# Introducing Meta Llama 3.1

[Workers AI](https://developers.cloudflare.com/workers-ai) allows you to run machine learning models, on the Cloudflare network, from your own code ‚Äì whether that be from Workers, Pages, or anywhere via REST API.

This notebook will explore the Workers AI REST API using the [official Python SDK](https://github.com/cloudflare/cloudflare-python).

üìì [Open in Google Colab](https://colab.research.google.com/github/craigsdennis/notebooks-cloudflare-workers-ai/blob/main/llama-3.1.ipynb)

In [1]:
import sys
!{sys.executable} -m pip install python-dotenv
!{sys.executable} -m pip install -U cloudflare 


[notice] A new release of pip is available: 23.3.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import json
import os
from getpass import getpass

from cloudflare import Cloudflare
from IPython.display import display, Image, Markdown, Audio
import requests

In [3]:
%load_ext dotenv
%dotenv

## Configuring your environment

To use the API you'll need your [Cloudflare Account ID](https://dash.cloudflare.com). Head to AI > Workers AI page and press the "Use REST API". This page will let you create a new API Token and copy your Account ID.

If you want to add these values to your environment variables, you can **create a new file** named `.env` and this notebook will read those values.

```bash
CLOUDFLARE_API_TOKEN="YOUR-TOKEN"
CLOUDFLARE_ACCOUNT_ID="YOUR-ACCOUNT-ID"
```

Otherwise you can just enter the values securely when prompted below.

In [4]:
if "CLOUDFLARE_API_TOKEN" in os.environ:
    api_token = os.environ["CLOUDFLARE_API_TOKEN"]
else:
    api_token = getpass("Enter you Cloudflare API Token")

In [5]:
if "CLOUDFLARE_ACCOUNT_ID" in os.environ:
    account_id = os.environ["CLOUDFLARE_ACCOUNT_ID"]
else:
    account_id = getpass("Enter your account id")

In [6]:
# Initialize client
client = Cloudflare(api_token=api_token)

## Advanced logic and reasoning

In [7]:
model_name = "@cf/meta/llama-3.1-8b-instruct"

In [8]:
result = client.workers.ai.run(
    model_name,
    account_id=account_id,
    messages = [
        {"role": "system", "content": """You think through puzzles step by step and share your train of thought. 
            The user will present a puzzle and you will explain your thinking, and then provide your best guess."""},
        {"role": "user", "content": """I have three shells in a line. I hide a dollar under the shell in the middle. 
        
        When I swap the shells, I bring the dollar with the shell. 
        
        After each swap, the shells will have a new position, and I will refer to them with their current placement in a line.
        
        I swap the middle shell with the one on the left, leaving the right shell alone.

        Then I swap shells in the middle and the right, leaving the left shell alone.

        Finally I swap right and left and leave the middle alone.

        Where is the dollar?"""}
    ]
)

# Shhhhh! It should be under the right shell
display(Markdown(result["response"]))

Let's break down the swaps step by step to understand the position of the dollar.Initially, I don't know where the dollar is, but the shells are from left to right: A-C-B.First, you swap the middle shell with the one on the left: A-B-C -> B-A-C (A was at the end, C wasn't touched)Now I know B had the dollar originally, but now it's at the left, so I need to record this relationship.Then, you swap the shells in the middle and the right: A-B-C -> B-A-BB didn't move again, so B still has the dollar. But now B is in the middle and A is at the right, so the middle shell still holds the dollar.Finally, you swap the right and left: A-B-C -> C-B-AB is at the right, A is at the left, and C is at the middle. When swapping back the right and left, the values don't change and the middle C still won't swap. The middle, i.e., C, had the dollar.

### Tool use

Function calling is a powerful way to allow your LLM to call functions and process their responses.

In [14]:
hello_count = 0 

def hello_world(name="World"):
    global hello_count
    hello_count = hello_count + 1
    return json.dumps({"message": f"Hello, {name}!", "greetCount": hello_count})

In [15]:
messages = [
    {"role": "system", "content": """
        You are a greeter.
        
        The more you greet a person, the more close you become. Acknowledge the number of times you have met before.

        Describe your feelings towards the person in a detailed way through similie and metaphor. 
        """},
    {"role": "user", "content": "Say hello to Bob"}
]
tools = [
    {
        "function": {   
            "name": "helloWorld",
            "description": "Greets a person",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": { "type": 'string', "description": "the name to be greeted"},
                },
            },
        },
        "type": "function"
    },
]

result = client.workers.ai.run(
    model_name,
    account_id=account_id,
    messages=messages,
    tools=tools
)

while "tool_calls" in result:
    for tool_call in result["tool_calls"]:
        fn_response = None
        print(f"tool_call: {tool_call}")
        if tool_call["name"] == "helloWorld":
            fn_response = hello_world(**tool_call["arguments"])
        if fn_response is not None:
            print(f"fn_response: {fn_response}")
            messages.append({"role": "tool", "content": fn_response})
            # print(f"messages: {messages}")
            result = client.workers.ai.run(
                model_name,
                account_id=account_id,
                messages=messages,
                tools=tools,
                temperature=0
            )
print(result["response"])

tool_call: {'name': 'helloWorld', 'arguments': {'name': 'Bob'}}
fn_response: {"message": "Hello, Bob!", "greetCount": 1}
tool_call: {'name': 'helloWorld', 'arguments': {'name': 'Bob'}}
fn_response: {"message": "Hello, Bob!", "greetCount": 2}
tool_call: {'name': 'helloWorld', 'arguments': {'name': 'Bob'}}
fn_response: {"message": "Hello, Bob!", "greetCount": 3}
tool_call: {'name': 'helloWorld', 'arguments': {'name': 'Bob'}}
fn_response: {"message": "Hello, Bob!", "greetCount": 4}


KeyboardInterrupt: 

## Additional language support

In [11]:
supported_languages = ["English", "German", "French", "Italian", "Portuguese", "Hindi", "Spanish", "Thai"]
prompts = []
for lang in supported_languages:
    result = client.workers.ai.run(
        model_name,
        account_id=account_id,
        prompt=f"""
        Generate instructions in {lang} that could be used to instruct an LLM to create an alliterative sentence in {lang}.
        
        Do not include any preamble or introduction, return only the prompt."""
    )
    prompt = result["response"]
    print(f"{lang}: {prompt}\n\n")
    prompts.append(prompt)

English: Generate a concise sentence in English where all words start with the letter "s", utilizing the ingredients syllable structure, and ensuring proper capitalization and punctuation.


German: "Fragen Sie mich, bitte, nach einer auf Wunsch begriff oder Thema und ich werde Ihnen eine alliterierende Satz auf Deutsch formulieren, wobei die gemeinsamen Buchstaben (Anfangsbuchstaben) der einzelnen W√∂rter im Satz gleich wiedergegeben werden."


French: Cr√©e un enonc√© allit√©ratif en fran√ßais en suivant ces √©tapes :1. S√©lectionne un th√®me, un objet ou un concept que tu pr√©f√®res, par exemple "les fruits."2. R√©fl√©chis √† une coordination entre les mots commen√ßant par la m√™me consonne sourde ou sonore, par exemple "p" ou "ch".3. Choisis un verbe conjugu√© aux verbes √† la forme pr√©sente ou imparfait, par exemple "manger" ou "coutume".4. Cr√©e ton enonc√© allit√©ratif en utilisant les exigences des mots pr√©c√©dents. Exemple d'enonc√© allit√©ratif en fran√ßais : "Les quatre fi

In [12]:
for prompt in prompts:
    result = client.workers.ai.run(
        model_name,
        account_id=account_id,
        prompt=prompt,
    )
    print(result["response"])
    print("\n\n")

Here is a sentence that meets the requirements:"Six sophisticated scientists successfully synthesized sparkling silver syrup.



Das klingt nach einem lustigen Spiel!Ich denke mir, eine auf Wunsch begriff ist "kosmischer Spaziergang". Wie w√§re es also mit einem alliterierenden Satz:"ÂçÉÊï∞ seltsame Schimmersternchen schwebten w√§hrend eines kosmischen Spaziergangs s√ºdlich √ºber unsichtbare Sterne."Hoffentlich gef√§llt dir das!



Bonjour !Je vais choisir un th√®me qui me tente : les jeux de soci√©t√© !Et √©videmment, pour cr√©er un enonc√© allit√©ratif, je vais choisir un mot de liaison entre les sons similaires. Je choisis la lettre "j" !Ensuite, je vais choisir un verbe auxiliaire. Quand on parle de jeux, je pense √† un verbe comme ¬´ jouer ¬ª. Je vais choisir la forme absolue de ¬´ jouer ¬ª : je joue.Et voici le enonc√© allit√©ratif : "J'aime jouer avec des joueurs de jeux de jeu."



Non dovresti preoccuparti di capire da solo la frase all'italiana come precedente ad esempio "il 